In [1]:
# # Run this and then restart the kernel at the start of each session to install
# # 'teotil3' in development mode
# !pip install -e /home/jovyan/projects/teotil3/

In [2]:
import os

import geopandas as gpd
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd
import seaborn as sn
import teotil3 as teo

plt.style.use("ggplot")

# TDP fractions

Replying to e-mail from Jon Lasse received 26.09.2024 at 14:52.

In [3]:
def get_teotil3_results_for_vassoms(st_yr, end_yr, vassom_list, agri_loss_model):
    """ """
    main_catches = [f"{i:03d}." for i in vassom_list]
    df = pd.read_csv(
        f"/home/jovyan/shared/common/teotil3/evaluation/teo3_results_nve2023_2013-2022_agri-{agri_loss_model}-loss.csv"
    )
    df = df.query(
        "(regine in @main_catches) and (year >= @st_yr) and (year <= @end_yr)"
    ).copy()
    df["År"] = df["year"]
    cols = [i for i in df.columns if i.split("_")[0] == "accum"]
    df = df[["regine", "År"] + cols]
    for col in df.columns:
        if col.endswith("_kg"):
            new_col = col.replace("_kg", "_tonnes")
            df[new_col] = df[col] / 1000
            del df[col]

    return df


def get_aggregation_dict_for_columns(par):
    """Make a dict mapping TEOTIL column names to summary columns with
    aggregation where necessary.

    Args
        par: Str. Either 'n' or 'p'

    Returns
        Dict with key's equal to output headings and values are lists
        of columns to aggregate.
    """
    if par.endswith("n"):
        agg_dict = {
            "Akvakultur": [f"accum_aquaculture_{par}_tonnes"],
            "Jordbruk": [f"accum_agriculture_{par}_tonnes"],
            "Avløp": [
                f"accum_large-wastewater_{par}_tonnes",
                f"accum_spredt_{par}_tonnes",
            ],
            "Industri": [f"accum_industry_{par}_tonnes"],
            "Bebygd": [f"accum_urban_{par}_tonnes"],
            "Bakgrunn": [
                f"accum_agriculture-background_{par}_tonnes",
                f"accum_upland_{par}_tonnes",
                f"accum_wood_{par}_tonnes",
                f"accum_lake_{par}_tonnes",
            ],
        }
    else:
        # No lake dep
        agg_dict = {
            "Akvakultur": [f"accum_aquaculture_{par}_tonnes"],
            "Jordbruk": [f"accum_agriculture_{par}_tonnes"],
            "Avløp": [
                f"accum_large-wastewater_{par}_tonnes",
                f"accum_spredt_{par}_tonnes",
            ],
            "Industri": [f"accum_industry_{par}_tonnes"],
            "Bebygd": [f"accum_urban_{par}_tonnes"],
            "Bakgrunn": [
                f"accum_agriculture-background_{par}_tonnes",
                f"accum_upland_{par}_tonnes",
                f"accum_wood_{par}_tonnes",
            ],
        }

    return agg_dict


def aggregate_parameters(df, par):
    """Aggregate columns in TEOTIL output to headings used in the report.

    Args
        df:    Dataframe of TEOTIL results
        par:   Str. Either 'n' or 'p'
        model: Str. Either 'teotil2' or 'teotil3'

    Returns
        Dataframe.
    """
    agg_dict = get_aggregation_dict_for_columns(par)
    for group, cols in agg_dict.items():
        df[group] = df[cols].sum(axis=1)

    df = df[["regine", "År"] + list(agg_dict.keys())]

    return df

In [4]:
teo3_df = get_teotil3_results_for_vassoms(2013, 2022, list(range(1, 18)), "annual")
totp_df = aggregate_parameters(teo3_df, "totp")
tdp_df = aggregate_parameters(teo3_df, "tdp")

In [5]:
out_df = pd.concat(
    [totp_df.sum(numeric_only=True), tdp_df.sum(numeric_only=True)], axis="columns"
)
out_df.columns = ["totp", "tdp"]
out_df["pct_tdp"] = 100 * out_df["tdp"] / out_df["totp"]
out_df

,totp,tdp,pct_tdp
År,342975.000000,342975.000000,100.000000
Akvakultur,0.027900,0.008100,29.032258
Jordbruk,5089.938585,1219.386450,23.956801
Avløp,1533.176516,527.005948,34.373469
Industri,292.769912,150.002439,51.235606
Bebygd,870.359039,609.568850,70.036482
Bakgrunn,1289.459818,640.409213,49.664922


In [12]:
df_dict = {"totp": [], "tdp": []}
for year in range(2013, 2023):
    csv_path = f"/home/jovyan/shared/common/teotil3/annual_input_data/agri_annual_loss/teotil3_input_data_nve2023_{year}.csv"
    df = pd.read_csv(csv_path)
    df["vassom"] = df["regine"].str.split(".", n=1).str[0].astype(int)
    df = df.query("1 <= vassom <= 17")
    cols = [col for col in df.columns if col.endswith("_kg")]
    df = df[cols]

    for par in ['totp', 'tdp']:
        agg_dict = {
            "Akvakultur": [f"aquaculture_{par}_kg"],
            "Jordbruk": [f"agriculture_{par}_kg"],
            "Avløp": [
                f"large-wastewater_{par}_kg",
                f"spredt_{par}_kg",
            ],
            "Industri": [f"industry_{par}_kg"],
            "Bebygd": [f"urban_{par}_kg"],
            "Bakgrunn": [
                f"agriculture-background_{par}_kg",
                f"upland_{par}_kg",
                f"wood_{par}_kg",
            ],
        }
        par_df = df.copy()
        for group, cols in agg_dict.items():
            par_df[group] = par_df[cols].sum(axis=1)
    
        par_df = par_df[list(agg_dict.keys())]

        df_dict[par].append(par_df)

totp_df = pd.concat(df_dict['totp'], axis='rows')
tdp_df = pd.concat(df_dict['tdp'], axis='rows')
in_df = pd.concat(
    [totp_df.sum(numeric_only=True), tdp_df.sum(numeric_only=True)], axis="columns"
) / 1000
in_df.columns = ["totp", "tdp"]
in_df["pct_tdp"] = 100 * in_df["tdp"] / in_df["totp"]
in_df

,totp,tdp,pct_tdp
Akvakultur,0.027900,0.008100,29.032258
Jordbruk,8551.010773,1497.900930,17.517238
Avløp,2042.943400,600.631600,29.400305
Industri,308.021900,154.010700,49.999919
Bebygd,1170.219300,702.135500,60.000335
Bakgrunn,3966.847255,940.292483,23.703773


In [13]:
out_df

,totp,tdp,pct_tdp
År,342975.000000,342975.000000,100.000000
Akvakultur,0.027900,0.008100,29.032258
Jordbruk,5089.938585,1219.386450,23.956801
Avløp,1533.176516,527.005948,34.373469
Industri,292.769912,150.002439,51.235606
Bebygd,870.359039,609.568850,70.036482
Bakgrunn,1289.459818,640.409213,49.664922


In [32]:
df_list = []
for year in range(2013, 2023):
    xls_path = f"/home/jovyan/shared/common/teotil3/agri_data/nibio_agri_data_{year}.xlsx"
    df = pd.read_excel(xls_path, sheet_name='Annual loss')
    df = df.dropna(subset="Regine unit")
    df["vassom"] = df["Regine unit"].str.split(".", n=1).str[0].astype(int)
    df = df.query("1 <= vassom <= 17")
    df_list.append(df)
    df['tdp'] = df['PO4-P netto annual loss'].astype(float)
    df['totp'] = df['TP netto annual loss'].astype(float)

    pct = 100 * df['tdp'].sum() / df['totp'].sum()
    print(f"{year}: {pct:.1f}%")

df = pd.concat(df_list, axis='rows')
df = df.sum(numeric_only=True)
df

2013: 17.8%
2014: 17.0%
2015: 17.5%
2016: 17.8%
2017: 17.8%
2018: 17.2%
2019: 17.7%
2020: 17.5%
2021: 17.9%
2022: 17.1%


vassom    2.485200e+05
tdp       1.497901e+06
totp      8.551011e+06
dtype: float64

In [29]:
df/1000

vassom     248.520000
tdp       1497.900930
totp      8551.010773
dtype: float64